In [1]:
import sqlite3

In [ ]:
# Define a connection and a cursor
# Connection represents the database
conn = sqlite3.connect('example.db') # can also use ':memory:'

# Cursor interacts w/ database through sql commands
cursor = conn.cursor()

# Creating table "employees" w/ 3 columns of specified data types
cursor.execute("""CREATE TABLE employees (
               first TEXT,
               last TEXT,
               pay INTEGER
               )""")
# conn.commit() # commit the transaction
# conn.close() # close the connection made to the database (-journal files will go away)

In [3]:
# Adding some data
cursor.execute("INSERT INTO employees VALUES ('Corey', 'Schafer', 50000)")

In [4]:
# Viewing a query
cursor.execute("SELECT * FROM employees WHERE last = 'Schafer'")
print(cursor.fetchone()) # grabs one from the table. There is also fetchall(), fetchmany(5), etc

('Corey', 'Schafer', 50000)


In [5]:
# Adding some data
cursor.execute("INSERT INTO employees VALUES ('Mary', 'Schafer', 70000)")

# Viewing again
cursor.execute("SELECT * FROM employees WHERE last = 'Schafer'")
print(cursor.fetchall())

[('Corey', 'Schafer', 50000), ('Mary', 'Schafer', 70000)]


### Use classes to create employees!

In [6]:
# Class that allows us to create employees
class Employee:
    def __init__(self, first, last, pay):
        self.first = first
        self.last = last
        self.pay = pay
    
    @property
    def email(self):
        return '{}.{}@email.com'.format(self.first, self.last)
    
    @property
    def fullname(self):
        return '{} {}'.format(self.first, self.last)
    
    def __repr__(self):
        return "Employee('{}', '{}', '{}')".format(self.first, self.last, self.pay)

### Now use that class to create employees and add them to the database!

In [11]:
# Now I can use the class to create new employees
emp_1 = Employee('John', 'Doe', 80000)
emp_2 = Employee('Jane', 'Doe', 90000)

# print(emp_1.first)
# print(emp_1.last)
# print(emp_1.pay)

# Adding them to the database
# WARNING: This method is vulnerable to sql injection attacks
### cursor.execute("INSERT INTO employees VALUES ('{}', '{}', '{}')".format(emp_1.first, emp_1.last, emp_1.pay))

#...so try it one of these two ways...
### cursor.execute("INSERT INTO employees VALUES (?, ?, ?)", (emp_1.first, emp_1.last, emp_1.pay))
### cursor.execute("INSERT INTO employees VALUES (?, ?, ?)", (emp_2.first, emp_2.last, emp_2.pay))

cursor.execute("INSERT INTO employees VALUES (:first, :last, :pay)",
               {'first': emp_1.first, 'last': emp_1.last, 'pay': emp_1.pay}) # dict keys will be first, last, and pay
cursor.execute("INSERT INTO employees VALUES (:first, :last, :pay)",
               {'first': emp_2.first, 'last': emp_2.last, 'pay': emp_2.pay})

# Viewing again
cursor.execute("SELECT * FROM employees")
print(cursor.fetchall())


[('Corey', 'Schafer', 50000), ('Mary', 'Schafer', 70000), ('John', 'Doe', 80000), ('Jane', 'Doe', 90000)]


In [ ]:
# You can also try viewing more specifically
cursor.execute("SELECT * FROM employees WHERE last=:last", {'last': 'Doe'})
print(cursor.fetchall())

[('John', 'Doe', 80000), ('Jane', 'Doe', 90000)]


In [15]:
conn.commit() # commit the transaction(s)
conn.close() # close the connection made to the database (-journal files will go away)

### Create functions to make the job easier!

In [16]:
# Create functions to do insertion/deleting/updating, etc. jobs!
def insert_emp(emp):
    with conn: # within contexts manager
        cursor.execute("INSERT INTO employees VALUES (:first, :last, :pay)",
                    {'first': emp.first, 'last': emp.last, 'pay': emp.pay})

def get_emps_by_name(lastname):
    cursor.execute("SELECT * FROM employees WHERE last=:last", {'last': lastname})
    return cursor.fetchall()

def update_pay(emp, pay):
    with conn:
        cursor.execute("""UPDATE employees SET pay = :pay
                       WHERE first = :first AND last = :last""",
                    {'first': emp.first, 'last': emp.last, 'pay': pay})

def remove_emp(emp):
    with conn:
        cursor.execute("DELETE FROM employees WHERE first = :first AND last = :last",
                    {'first': emp.first, 'last': emp.last})

### Testing out the new functions!

In [18]:
conn = sqlite3.connect('example.db') # reopening the connection
cursor = conn.cursor()

print(get_emps_by_name('Doe'))

[('John', 'Doe', 80000), ('Jane', 'Doe', 90000)]


In [19]:
update_pay(emp_2, 95000) # updating Jane's pay
remove_emp(emp_1) # removing John
print(get_emps_by_name('Doe'))

[('Jane', 'Doe', 95000)]


In [23]:
conn.close()